In [22]:
import numpy as np
from random import uniform
import sys

## Functions and derivatives

In [23]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def dsigmoid(y):
    return y * (1 - y)


def dtanh(x):
    return 1 - x * x


# The numerically stable softmax implementation
def softmax(x):
    # assuming x shape is [feature_size, batch_size]
    e_x = np.exp(x - np.max(x, axis=0))
    return e_x / e_x.sum(axis=0)

## Data I/O

In [24]:
# data I/O
data = open("data/input.txt", "r").read()  # should be simple plain text file
chars = sorted(list(set(data)))
data_size, vocab_size = len(data), len(chars)
print("data has %d characters, %d unique." % (data_size, vocab_size))
char_to_ix = {ch: i for i, ch in enumerate(chars)}
ix_to_char = {i: ch for i, ch in enumerate(chars)}
std = 0.1

data has 1115394 characters, 65 unique.


## Hyperparameters

In [71]:
emb_size = 16
hidden_size = 256  # size of hidden layer of neurons
seq_length = 128  # number of steps to unroll the RNN for
learning_rate = 5e-2
max_updates = 500000
batch_size = 32

concat_size = emb_size + hidden_size

## Model parameters

In [80]:
dim_check = True

In [26]:
# model parameters
# char embedding parameters
Wex = np.random.randn(emb_size, vocab_size) * std  # embedding layer

# LSTM parameters
Wf = np.random.randn(hidden_size, concat_size) * std  # forget gate
Wi = np.random.randn(hidden_size, concat_size) * std  # input gate
Wo = np.random.randn(hidden_size, concat_size) * std  # output gate
Wc = np.random.randn(hidden_size, concat_size) * std  # c term

bf = np.zeros((hidden_size, 1))  # forget bias
bi = np.zeros((hidden_size, 1))  # input bias
bo = np.zeros((hidden_size, 1))  # output bias
bc = np.zeros((hidden_size, 1))  # memory bias

# Output layer parameters
Why = np.random.randn(vocab_size, hidden_size) * std  # hidden to output
by = np.random.randn(vocab_size, 1) * std  # output bias

In [27]:
data_stream = np.asarray([char_to_ix[char] for char in data])
print(data_stream.shape)

bound = (data_stream.shape[0] // (seq_length * batch_size)) * (seq_length * batch_size)
cut_stream = data_stream[:bound]
cut_stream = np.reshape(cut_stream, (batch_size, -1))

(1115394,)


## Forward

In [89]:
def forward(inputs, targets, memory):
    """
    inputs,targets are both list of integers.
    hprev is Hx1 array of initial hidden state
    returns the loss, gradients on model parameters, and last hidden state
    """
    hprev, cprev = memory
    
    xs = {} # input
    wes = {} # word embeddings
    hs = {} # hidden states
    zs = {} # concat of last hidden state and input
    cs = {} # cell states
    fs = {} # forget gates
    ins = {} # input gates
    c_s = {} # candidate cell states
    os = {} # output gates
    ys = {} # unnormalized output (before softmax)
    ps = {} # normalized probability output (after softmax)
    ls = {} # labels (ground truth)
    
    hs[-1] = np.copy(hprev)
    cs[-1] = np.copy(cprev)

    loss = 0
    input_length = inputs.shape[0]

    # forward pass
    for t in range(input_length):
        xs[t] = np.zeros((vocab_size, batch_size))  # encode in 1-of-k representation
        for b in range(batch_size):
            xs[t][inputs[t][b]][b] = 1

        # convert word indices to word embeddings
#         wes[t] = np.dot(Wex, xs[t])
        wes[t] = Wex @ xs[t]

        # LSTM cell operation
        # first concatenate the input and h to get z
        # Note: Use np.vstack instead of np.row_stack
        zs[t] = np.vstack((hs[t - 1], wes[t]))
        

        # compute the forget gate
        # f = sigmoid(Wf * z + bf)
        fs[t] = sigmoid(Wf @ zs[t] + bf)
        

        # compute the input gate
        # i = sigmoid(Wi * z + bi)
        ins[t] = sigmoid(Wi @ zs[t] + bi)
        
        # compute the candidate memory
        # c_ = tanh(Wc * z + bc)
        c_s[t] = np.tanh(Wc @ zs[t] + bc)
        

        # new memory: applying forget gate on the previous memory
        # and then adding the input gate on the candidate memory
        cs[t] = fs[t] * cs[t-1] + ins[t] * c_s[t]
        

        # output gate
        # o = sigmoid(Wo * z + bo)
        os[t] = sigmoid(Wo @ zs[t] + bo)
        

        # hidden state
        hs[t] = os[t] * np.tanh(cs[t])
        
        
        # output layer (unnormalized)
        ys[t] = Why @ hs[t] + by
        
        
        # Softmax layer to get normalized probabilities
        ps[t] = softmax(ys[t])
        


        # label
        ls[t] = np.zeros((vocab_size, batch_size))
        for b in range(batch_size):
            ls[t][targets[t][b]][b] = 1

        # cross-entropy loss
        loss_t = np.sum(-np.log(ps[t]) * ls[t])
        loss += loss_t
        # loss += -np.log(ps[t][targets[t],0])

#         if dim_check:
#             print("z:", zs[t].shape)
#             print("f:", fs[t].shape)
#             print("i:", ins[t].shape)
#             print("c_:", c_s[t].shape)
#             print("c:", cs[t].shape)
#             print("o:", os[t].shape)
#             print("h:", hs[t].shape)
#             print("ys[t]:", ys[t].shape)
#             print("p:", ps[t].shape)
#             print("l:", ls[t].shape)
#             print("loss:", loss_t.shape)
#             print("-------------")
        
    activations = (xs, wes, hs, ys, ps, cs, zs, ins, c_s, ls, os, fs)
    memory = (hs[input_length - 1], cs[input_length - 1])

    return loss, activations, memory


## Backward

In [85]:
def backward(activations, clipping=True):
    xs, wes, hs, ys, ps, cs, zs, ins, c_s, ls, os, fs = activations

    # backward pass: compute gradients going backwards
    # Here we allocate memory for the gradients
    dWex, dWhy = np.zeros_like(Wex), np.zeros_like(Why)
    dby = np.zeros_like(by)
    dWf, dWi, dWc, dWo = np.zeros_like(Wf), np.zeros_like(Wi), np.zeros_like(Wc), np.zeros_like(Wo)
    dbf, dbi, dbc, dbo = np.zeros_like(bf), np.zeros_like(bi), np.zeros_like(bc), np.zeros_like(bo)

    dhnext = np.zeros_like(hs[0])
    dcnext = np.zeros_like(cs[0])

    input_length = len(xs)

    # back propagation through time starts here
    for t in reversed(range(input_length)):
        
        # Softmax loss gradient
        dy = ls[t] - ys[t]
        
        
        # Note:
        # About matrix transpose in gradient, check out
        # https://cs231n.github.io/optimization-2/#staged
        
        # Hidden to output gradient
        # Gradient for ys[t] = Why @ hs[t] + by
        dWhy = dy @ (hs[t].T)
        dby = dy[:, -1]
        dh = (Why.T) @ dy + dhnext # need to add gradients from the next hidden state
        
        
        # Gradient for hs[t] = os[t] * np.tanh(cs[t])
        ## Gradient for os[t] 
        do = np.tanh(cs[t]) * dh
#         do = dsigmoid(os[t]) * do
        ## Gradient for cs[t]
        dc = os[t] * dh * dtanh(cs[t])
        dc = dc + dcnext
        
        
        # Gradient for cs[t] = fs[t] * cs[t-1] + ins[t] * c_s[t]
        ## Gradient for fs[t]
        df = cs[t - 1] * dc
#         df = dsigmoid(fs[t]) * df
        ## Gradient for ins[t]
        di = c_s[t] * dc
#         di = dsigmoid(ins[t]) * di
        ## Gradient for c_s[t]
        dc_ = ins[t] * dc
#         dc_ = dtanh(c_s[t]) * dc_
        
    
        # Gate gradients
        ## Forget gate
        ## Gradients for fs[t] = sigmoid(Wf @ zs[t] + bf)
        df = dsigmoid(fs[t]) * df
        dWf = df @ (zs[t].T)
        dbf = df
        dzf = Wf.T @ df

        ## Input gate
        ## Gradients for ins[t] = sigmoid(Wi @ zs[t] + bi)
        di = dsigmoid(ins[t]) * di
        dWi = di @ (zs[t].T)
        dbi = di
        dzi = Wi.T @ di 

        ## Output gate
        ## Gradients for s[t] = sigmoid(Wo @ zs[t] + bo)
        do = dsigmoid(os[t]) * do
        dWo = do @ (zs[t].T)
        dbo = do
        dzo = Wo.T @ do
        
        ## Candidate cell state
        ## Gradients for c_s[t] = np.tanh(Wc @ zs[t] + bc)
        dc_ = dtanh(c_s[t]) * dc_
        dWc = dc_ @ (zs[t].T)
        dbc = dc_
        dzc = Wc.T @ dc_
        
        # zs[t] (concat of hs[t-1] and wes[t]) is used in multiple gates, 
        # thus the gradient must be accumulated
        dz = dzo + dzc + dzi + dzf
        
        # Gradient for hs[t-1]
        dhnext = dz[:hidden_size, :]
        
        # Gradient for cs[t-1] in cs[t] = fs[t] * cs[t-1] + ins[t] * c_s[t]
        dcnext = fs[t] * dc
        
        if dim_check:
            print("dy:", dy.shape)
            
            print("dWhy:", dWhy.shape)
            print("dby:", dby.shape)
            print("dh:", dh.shape)
            
            print("-------------")
            
            
            
            
        

    # clip to mitigate exploding gradients
    if clipping:
        for dparam in [dWex, dWf, dWi, dWo, dWc, dbf, dbi, dbo, dbc, dWhy, dby]:
            np.clip(dparam, -5, 5, out=dparam)

    gradients = (dWex, dWf, dWi, dWo, dWc, dbf, dbi, dbo, dbc, dWhy, dby)

    return gradients

## Sample

In [32]:
def sample(memory, seed_ix, n):
    """
    sample a sequence of integers from the model
      h is memory state, seed_ix is seed letter for first time step
    """
    h, c = memory
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []
    for t in range(n):

        # forward pass again, but we do not have to store the activations now
        
        # Embedding of input x
        wes = Wex @ x
        
        # Concate hidden state and embedding
        z = np.vstack((h, wes))
        
        # Forget gate
        f =  sigmoid(Wf @ z + bf)
        
        # Input gate
        i = sigmoid(Wi @ z + bi)
        
        # Candidate cell state
        c_ = np.tanh(Wc @ z + bc)
        
        # New cell state
        c = f * c + i * c_
        
        # Output gate
        o = sigmoid(Wo @ z + bo)
        
        # New hidden state
        h = o * np.tanh(c)
        
        # output layer (unnormalized)
        y = Why @ h + by
        
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_size), p=p.ravel())

        index = ix
        x = np.zeros((vocab_size, 1))
        x[index] = 1
        ixes.append(index)
    return ixes

## Train

In [90]:
n, p = 0, 0
n_updates = 0

# momentum variables for Adagrad
mWex, mWhy = np.zeros_like(Wex), np.zeros_like(Why)
mby = np.zeros_like(by)

mWf, mWi, mWo, mWc = np.zeros_like(Wf), np.zeros_like(Wi), np.zeros_like(Wo), np.zeros_like(Wc)
mbf, mbi, mbo, mbc = np.zeros_like(bf), np.zeros_like(bi), np.zeros_like(bo), np.zeros_like(bc)

smooth_loss = -np.log(1.0 / vocab_size) * seq_length  # loss at iteration 0

data_length = cut_stream.shape[1]

while n < 10:
    # prepare inputs (we're sweeping from left to right in steps seq_length long)
    if p + seq_length + 1 >= data_length or n == 0:
        hprev = np.zeros((hidden_size, batch_size))  # reset RNN memory
        cprev = np.zeros((hidden_size, batch_size))
        p = 0  # go from start of data

    inputs = cut_stream[:, p : p + seq_length].T
    targets = cut_stream[:, p + 1 : p + 1 + seq_length].T

    # sample from the model now and then
#     if n % 200 == 0:
#         h_zero = np.zeros((hidden_size, 1))  # reset RNN memory
#         c_zero = np.zeros((hidden_size, 1))
#         sample_ix = sample((h_zero, c_zero), inputs[0][0], 2000)
#         txt = "".join(ix_to_char[ix] for ix in sample_ix)
#         print("----\n %s \n----" % (txt,))

    # forward seq_length characters through the net and fetch gradient
    loss, activations, memory = forward(inputs, targets, (hprev, cprev))
    hprev, cprev = memory
    gradients = backward(activations)

    dWex, dWf, dWi, dWo, dWc, dbf, dbi, dbo, dbc, dWhy, dby = gradients
    smooth_loss = smooth_loss * 0.999 + loss / batch_size * 0.001
    if n % 20 == 0:
        print("iter %d, loss: %f" % (n, smooth_loss))  # print progress

    # perform parameter update with Adagrad
    for index, (param, dparam, mem) in enumerate(zip(
        [Wf, Wi, Wo, Wc, bf, bi, bo, bc, Wex, Why, by],
        [dWf, dWi, dWo, dWc, dbf, dbi, dbo, dbc, dWex, dWhy, dby],
        [mWf, mWi, mWo, mWc, mbf, mbi, mbo, mbc, mWex, mWhy, mby],
    )):
        print(index)
        print("param:", param.shape)
        print("dparam:", dparam.shape)
        print("mem:", mem.shape)
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8)  # adagrad update

    p += seq_length  # move data pointer
    n += 1  # iteration counter
    n_updates += 1
    if n_updates >= max_updates:
        break


dy: (65, 32)
dWhy: (65, 256)
dby: (65, 32)
dh: (256, 32)
-------------
dy: (65, 32)
dWhy: (65, 256)
dby: (65, 32)
dh: (256, 32)
-------------
dy: (65, 32)
dWhy: (65, 256)
dby: (65, 32)
dh: (256, 32)
-------------
dy: (65, 32)
dWhy: (65, 256)
dby: (65, 32)
dh: (256, 32)
-------------
dy: (65, 32)
dWhy: (65, 256)
dby: (65, 32)
dh: (256, 32)
-------------
dy: (65, 32)
dWhy: (65, 256)
dby: (65, 32)
dh: (256, 32)
-------------
dy: (65, 32)
dWhy: (65, 256)
dby: (65, 32)
dh: (256, 32)
-------------
dy: (65, 32)
dWhy: (65, 256)
dby: (65, 32)
dh: (256, 32)
-------------
dy: (65, 32)
dWhy: (65, 256)
dby: (65, 32)
dh: (256, 32)
-------------
dy: (65, 32)
dWhy: (65, 256)
dby: (65, 32)
dh: (256, 32)
-------------
dy: (65, 32)
dWhy: (65, 256)
dby: (65, 32)
dh: (256, 32)
-------------
dy: (65, 32)
dWhy: (65, 256)
dby: (65, 32)
dh: (256, 32)
-------------
dy: (65, 32)
dWhy: (65, 256)
dby: (65, 32)
dh: (256, 32)
-------------
dy: (65, 32)
dWhy: (65, 256)
dby: (65, 32)
dh: (256, 32)
-------------
dy: (6

ValueError: non-broadcastable output operand with shape (256,1) doesn't match the broadcast shape (256,32)

## Gradient Check

In [21]:
data_length = cut_stream.shape[1]

p = 0
# inputs = [char_to_ix[ch] for ch in data[p:p + seq_length]]
# targets = [char_to_ix[ch] for ch in data[p + 1:p + seq_length + 1]]
inputs = cut_stream[:, p : p + seq_length].T
targets = cut_stream[:, p + 1 : p + 1 + seq_length].T

delta = 0.0001

hprev = np.zeros((hidden_size, batch_size))
cprev = np.zeros((hidden_size, batch_size))

memory = (hprev, cprev)

loss, activations, hprev = forward(inputs, targets, memory)
gradients = backward(activations, clipping=False)
dWex, dWf, dWi, dWo, dWc, dbf, dbi, dbo, dbc, dWhy, dby = gradients

for weight, grad, name in zip(
    [Wf, Wi, Wo, Wc, bf, bi, bo, bc, Wex, Why, by],
        [dWf, dWi, dWo, dWc, dbf, dbi, dbo, dbc, dWex, dWhy, dby],
        ["Wf", "Wi", "Wo", "Wc", "bf", "bi", "bo", "bc", "Wex", "Why", "by"],
    ):

        str_ = "Dimensions dont match between weight and gradient %s and %s." % (weight.shape, grad.shape)
        assert weight.shape == grad.shape, str_

        print(name)
        countidx = 0
        gradnumsum = 0
        gradanasum = 0
        relerrorsum = 0
        erroridx = []

        for i in range(weight.size):

            # evaluate cost at [x + delta] and [x - delta]
            w = weight.flat[i]
            weight.flat[i] = w + delta
            loss_positive, _, _ = forward(inputs, targets, memory)
            weight.flat[i] = w - delta
            loss_negative, _, _ = forward(inputs, targets, memory)
            weight.flat[i] = w  # reset old value for this parameter
            # fetch both numerical and analytic gradient
            grad_analytic = grad.flat[i]
            grad_numerical = (loss_positive - loss_negative) / (2 * delta)
            gradnumsum += grad_numerical
            gradanasum += grad_analytic
            rel_error = abs(grad_analytic - grad_numerical) / abs(grad_numerical + grad_analytic)
            if rel_error is None:
                rel_error = 0.0
            relerrorsum += rel_error

            if rel_error > 0.001:
                print("WARNING %f, %f => %e " % (grad_numerical, grad_analytic, rel_error))
                countidx += 1
                erroridx.append(i)

        print(
            "For %s found %i bad gradients; with %i total parameters in the vector/matrix!"
            % (name, countidx, weight.size)
        )
        print(
            " Average numerical grad: %0.9f \n Average analytical grad: %0.9f \n Average relative grad: %0.9f"
            % (gradnumsum / float(weight.size), gradanasum / float(weight.size), relerrorsum / float(weight.size))
        )
        print(" Indizes at which analytical gradient does not match numerical:", erroridx)

KeyError: 0